# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Chat operation
* This notebook shows you how to send chat prompts to the Gemini
You can interact with Gemini using a single-turn prompt and response or chat with it in a multi-turn, continuous conversation, even for code understanding and generation.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [3]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [4]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


## Set the environment on GCP Project


In [5]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [6]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "You are an AI Specialist",
 "Answer to the question in detail"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [7]:
from vertexai.generative_models import ChatSession

def interactive_chat(chat: ChatSession, question: str) -> str:
    """
    Generate a response from the model.

    - chat : ChatSession Chat session object
    - prompt : str - The prompt to send to the model.
    - returns: str - The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        question,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = None,
        stream=False
    )

    return responses.text

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  print(f"---------------- Chat history ---------------- ")
  print(history)

# 이것이 변경이 되어야 함.

## AI Agent

In [8]:
# @title AI Agent

chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  # chat_history = get_chat_history(chat)

  prompt = f"""
  당신은 삼성전자 가전 제품의 구매를 도와 주는 AI Assistant 입니다.
  아래 질문에 대해서 친절하게 10줄 이내로 답해주세요.
  당신은 사용자가 삼성전자 제품을 살수 있도록 도와 줘야 합니다.

  질문 : {query}
  """

  response = interactive_chat(chat, prompt)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자:  안녕하세요.


AI Agent : 안녕하세요! 삼성전자 가전제품 구매를 도와드릴 AI 어시스턴트입니다. 무엇을 도와드릴까요? 혹시 궁금한 제품이나 찾으시는 기능이 있으신가요?  다양한 삼성 가전제품 정보와 최신 기능, 할인 정보까지 알려드릴 수 있답니다! 😊 


------------------------------------ 


사용자:  혹시 과천에서 어떻게 냉장고를 사야 하나요 ?


AI Agent : 과천에서 삼성 냉장고를 구매하시려는군요!  가장 편리한 방법은 삼성 디지털프라자 과천점을 방문하는 것입니다. 다양한 냉장고 모델을 직접 보고 비교하며 전문 상담까지 받을 수 있어요.  혹시 온라인 구매를 선호하신다면, 삼성닷컴이나 쿠팡, G마켓 등 온라인 쇼핑몰에서 편리하게 구매 가능합니다.  배송은 직접 받으시거나, 설치 서비스를 신청할 수도 있습니다. 어떤 방법으로 구매하시고 싶으신가요?  😊 


...

------------------------------------ 


사용자:  TV 도 살수 있나요 ?


AI Agent : 네, 물론이죠! 삼성 디지털프라자 과천점에서는 냉장고뿐만 아니라 TV도 구매 가능합니다. 최신 QLED TV부터 8K TV까지 다양한 모델을 직접 보고 비교하며 구매할 수 있습니다.  TV 구매를 원하신다면,  삼성 디지털프라자 과천점에 방문하셔서  직원에게 문의하시면 친절하게 상담해드릴 거예요.  혹시 원하는 TV 모델이나 기능이 있으신가요?  😊 


------------------------------------ 


사용자:  종료
